# Abgabe 3 Mark Nagengast Porro, Tim Schwabe

In [111]:
import numpy as np

## Aufgabe 1

In [112]:
data = dict()
data.update([((0,0),0),((0,1),1),((1,0),1),((1,1),0)])

In [113]:
def RELU(x):
    return max(0,x)

In [114]:
def sigmoid(x):
    return 1/(1+(np.math.e)**(-x))

In [115]:
def createTrainingData(batch_size):

    data = np.array([[[0,0],0],
                     [[0,1],1],
                     [[1,0],1],
                     [[1,1],0]])

    traing_data = []
    target = []
    while(batch_size > 0):
        index = np.random.randint(4)
        traing_data.append( data[index][0])
        target.append(data[index][1])
        batch_size -= 1
    return np.array(traing_data), np.array(target)

train, target = createTrainingData(10)
print(train)
print(target)

[[1 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]]
[1 0 0 0 0 1 0 0 0 0]


c:\Users\timds\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


## Aufgabe 2

In [116]:
class NN():

    def __init__(self, training_data, target_data,hidden_layers):

        self.training_data = training_data
        self.target_data = target_data

        self.weights = [np.matrix(np.random.uniform(size=(2,4)))]
        while(hidden_layers > 1):
            self.weights.append(np.matrix(np.random.uniform(size=(4,16))))
            hidden_layers -=1
        self.weights.append(np.matrix(np.random.uniform(size=(4,1))))
        print(self.weights)


    def ReLU(x):
        return max(0,x)
    
    def sigmoid(x):
        return 1/(1+(np.math.e)**(-x))
    
    def forward():
        return

test = NN(train,target,1)

[matrix([[0.65967788, 0.48196294, 0.09416714, 0.54920499],
        [0.21519469, 0.4540999 , 0.79808083, 0.46653905]]), matrix([[0.78927321],
        [0.095511  ],
        [0.13264234],
        [0.1184199 ]])]
